In [ ]:
# Dependencies
!pip install oracledb
!pip install PyPDF2
!pip install langchain_huggingface
!pip install langchain_community


In [ ]:
# Imports
from PyPDF2 import PdfReader
import oracledb
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores.oraclevs import OracleVS
from langchain_community.embeddings import OCIGenAIEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_core.documents import BaseDocumentTransformer,Document
from google.colab import userdata

print("successfully imported libraries")

successfully imported libraries


In [ ]:
# Connect to Oracle a3ai
# from google.colab import files
# files.upload()   # choose Wallet_IH9O4WSS1Y1QW6CX.zip

# !unzip Wallet_IH9O4WSS1Y1QW6CX.zip -d wallet
# !pip install oracledb --upgrade

# Connection parameters
username = userdata.get('ORACLE_USERNAME')
password = userdata.get('ORACLE_PASSWORD')
walletPassword = userdata.get('ORACLE_WALLET_PASSWORD')
dsn = "ih9o4wss1y1qw6cx_high"  #DNS from wallet/tnsnames.ora

try:
    conn = oracledb.connect(
        user=username,
        password=password,
        dsn=dsn,
        config_dir="/content/wallet",  # Wallet folder
        wallet_location="/content/wallet",
        wallet_password=walletPassword
    )
    print("Connection successful!")
except oracledb.Error as e:
    print(f"Could not connect to the database - Error occurred: {str(e)}")

Connection successful!


In [ ]:
# Load the confluence solution document
with open("/content/solution_document.txt", 'r') as file:
        content = file.read()


In [ ]:
# Chunk the text document into smaller chunks
text_splitter = CharacterTextSplitter(separator=".",chunk_size=100,chunk_overlap=20)
chunks = text_splitter.split_text(content)

In [ ]:
# Chunks to document wrapper (suitable to save in oracle vector store )
def chunks_to_doc_wrapper(row:dict)-> Document :
  metadata = {'id': row['id'],'link': row['link']}
  return Document(page_content=row['text'],metadata = metadata)

In [ ]:
# Converting chunks to documemt using above method
docs = [chunks_to_doc_wrapper({'id': str(page_num), 'link': f'Page{page_num}', 'text':text}) for page_num, text in enumerate(chunks)]


In [ ]:
# Creating Embedding Model
#COMPARTMENT_OCID = ""
#embed_model = OCIGenAIEmbeddings(model_id="cohere.embed-english-v3.0",service_endpoint="",compartment_id=COMPARTMENT_OCID)

embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
# Configure the Oracle 23Ai Vector Store with Embedding model
knowledge_base = OracleVS.from_documents(docs,embed_model,client=conn,table_name="Company_Solutions",distance_strategy=DistanceStrategy.DOT_PRODUCT)

In [ ]:
print(knowledge_base)